**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
!pip install scikit-video
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
from numpy import random
import warnings
warnings.simplefilter("ignore")

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization, Flatten, Dropout


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act returns what action the agent will choose based on the current state. 

The epsilon parameter allows to control the exploration, this what we call the $\epsilon$-greedy strategy. The epsilon parameter is essential because if epsilon = 0 we will follow the action based on what we have learned only ie the strategy with the best reward till now. 

With the function act, the actor chooses a random action with probability $\epsilon$ and act via its policy with a probability $1-\epsilon$. 

It's a pramater of a trade-off exploration-exploitation.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

* The array ``board`` defines the distribution of the rewards over the $N\times N$ grid. It stores the reward of being in each cell that we can access through the coordinates x and y of the agent. 

* The array ``position`` defines the borders of the grid, the allowed states and the position of the agent. At each step, a value of -1 is assigned for the cells representing the borders (ie the states that can't be visited), a value of 0 is assigned for all the other cells that can be visited, and a value of 1 is assigned for the current cell occupied by the agent.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()

    def learned_act(self, s):
        return np.random.randint(0, self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        
        state = env.reset()
        game_over = False # We know that the games will end in infinity

        win, lose = 0, 0
        
        while not game_over:
            
            action = agent.learned_act(state)
            state,reward,game_over = env.act(action)

            if reward > 0:
                win += reward 
            if reward < 0:
                lose -= reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.0/17.0. Average score (-7.0)
Win/lose count 9.5/15.0. Average score (-6.25)
Win/lose count 7.0/16.0. Average score (-7.166666666666667)
Win/lose count 8.5/12.0. Average score (-6.25)
Win/lose count 8.5/12.0. Average score (-5.7)
Win/lose count 13.5/24.0. Average score (-6.5)
Win/lose count 12.5/16.0. Average score (-6.071428571428571)
Win/lose count 9.0/7.0. Average score (-5.0625)
Win/lose count 10.5/21.0. Average score (-5.666666666666667)
Win/lose count 9.5/9.0. Average score (-5.05)
Final score: -5.05


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




1 - We have:$$
\begin{align*} 
Q^\pi(s,a)&=E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
          &=E_{p^{\pi}}[r(s_0,a_0) + \gamma\sum_{t=1}^{\infty}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
          &=r(s,a) + \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a]\\
          &=r(s,a) + E_{(s',a')\sim p(.|s,a)}\left(\gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']\right)\\
          &=E_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']\right)\\
          &=E_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma Q^{\pi}(s',a') \right)
\end{align*}
$$

2 - We have $\pi^*(s') = arg\max_{a'} Q^{*}(s',a')$, which is deterministic, so:$$
\begin{align*}
Q^*(s,a)&= E_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma Q^{*}(s',a') \right)\\
        &= E_{s'\sim p(.|s,a)\,and\,a'\sim \pi^*(.|s')}\left(r(s,a) + \gamma Q^{*}(s',a') \right)\\
        &= E_{s' \sim \pi^*(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q^{*}(s',a') \right)\\
\end{align*}
$$

3 - We know that $$Q^*(s,a)=E_{s' \sim \pi^*(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q^{*}(s',a') \right)$$

Therefore, $Q^*$ is the solution of the equation $F(Q) = 0$ with:
$$F(Q) = E_{s' \sim \pi^*(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q(s',a') - Q(s,a)\right)$$

So minimizing $\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$ will lead us to find $Q^*$.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) <= self.max_memory:
            self.memory.append(m)
        else:
            del self.memory[0]

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]        

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):

    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        self.grid_size = grid_size
        
        # Number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1, s.shape[0], s.shape[1], s.shape[2]]))[0, :])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        on_batch = min(self.batch_size, len(self.memory.memory))
        input_states = np.zeros((on_batch, 5, 5, self.n_state))  
        target_q = np.zeros((on_batch, self.n_action))           
        
        for i in range(on_batch):
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access() 
            target_q[i] = self.model.predict(s_.reshape(1, 5, 5, self.n_state))
            input_states[i] = s_
            
            if game_over_:
                target_q[i, a_] = r_          
            else:
                target_q[i, a_] = r_ + self.discount*self.model.predict(n_s_.reshape(1, 5, 5, self.n_state)).max()
                
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(30,activation ='relu'))
        model.add(Dense(4, activation = None))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model


In [0]:
epochs_train = 40
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=1e-1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/040 | Loss 0.0117 | Win/lose count 3.5/8.0 (-4.5)
Epoch 001/040 | Loss 0.0074 | Win/lose count 4.0/4.0 (0.0)
Epoch 002/040 | Loss 0.0071 | Win/lose count 2.5/7.0 (-4.5)
Epoch 003/040 | Loss 0.0079 | Win/lose count 1.5/3.0 (-1.5)
Epoch 004/040 | Loss 0.0060 | Win/lose count 3.5/2.0 (1.5)
Epoch 005/040 | Loss 0.0038 | Win/lose count 5.0/6.0 (-1.0)
Epoch 006/040 | Loss 0.0134 | Win/lose count 5.0/3.0 (2.0)
Epoch 007/040 | Loss 0.0098 | Win/lose count 5.0/4.0 (1.0)
Epoch 008/040 | Loss 0.0056 | Win/lose count 6.5/6.0 (0.5)
Epoch 009/040 | Loss 0.0055 | Win/lose count 3.5/3.0 (0.5)
Epoch 010/040 | Loss 0.0045 | Win/lose count 4.5/3.0 (1.5)
Epoch 011/040 | Loss 0.0085 | Win/lose count 5.5/3.0 (2.5)
Epoch 012/040 | Loss 0.0090 | Win/lose count 5.5/5.0 (0.5)
Epoch 013/040 | Loss 0.0104 | Win/lose count 5.0/4.0 (1.0)
Epoch 014/040 | Loss 0.0051 | Win/lose count 5.5/0 (5.5)
Epoch 015/040 | Loss 0.0697 | Win/lose count 2.5/2.0 (0.5)
Epoch 016/040 | Loss 0.0033 | Win/lose count 8.5/4.0 (

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
from keras import regularizers as reg

class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        # First model
        # model = Sequential()
        # model.add(Conv2D(8, (3, 3), activation='relu'))
        # model.add(Conv2D(16, (2, 2), activation='relu'))
        # model.add(Flatten())
        # model.add(Dense(4, activation='linear'))

        # Final model
        model = Sequential()
        model.add(Conv2D(50, (2,2), input_shape=(5, 5, self.n_state,), activation='relu'))
        model.add(Conv2D(30, (2,2), activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))

        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/040 | Loss 0.0052 | Win/lose count 4.0/7.0 (-3.0)
Epoch 001/040 | Loss 0.0062 | Win/lose count 3.5/1.0 (2.5)
Epoch 002/040 | Loss 0.0030 | Win/lose count 4.0/2.0 (2.0)
Epoch 003/040 | Loss 0.0059 | Win/lose count 4.0/2.0 (2.0)
Epoch 004/040 | Loss 0.0014 | Win/lose count 4.0/4.0 (0.0)
Epoch 005/040 | Loss 0.0054 | Win/lose count 5.0/4.0 (1.0)
Epoch 006/040 | Loss 0.0039 | Win/lose count 9.0/4.0 (5.0)
Epoch 007/040 | Loss 0.0540 | Win/lose count 6.0/2.0 (4.0)
Epoch 008/040 | Loss 0.0043 | Win/lose count 2.5/4.0 (-1.5)
Epoch 009/040 | Loss 0.0040 | Win/lose count 5.5/2.0 (3.5)
Epoch 010/040 | Loss 0.0016 | Win/lose count 2.5/3.0 (-0.5)
Epoch 011/040 | Loss 0.0045 | Win/lose count 5.5/6.0 (-0.5)
Epoch 012/040 | Loss 0.1061 | Win/lose count 10.0/3.0 (7.0)
Epoch 013/040 | Loss 0.0005 | Win/lose count 11.5/2.0 (9.5)
Epoch 014/040 | Loss 0.0596 | Win/lose count 10.5/2.0 (8.5)
Epoch 015/040 | Loss 0.0012 | Win/lose count 3.5/1.0 (2.5)
Epoch 016/040 | Loss 0.0011 | Win/lose count 4.0/

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 8.0/0. Average score (8.0)
Win/lose count 1.0/0. Average score (4.5)
Win/lose count 4.0/0. Average score (4.333333333333333)
Win/lose count 8.0/0. Average score (5.25)
Win/lose count 4.0/0. Average score (5.0)
Win/lose count 1.0/0. Average score (4.333333333333333)
Win/lose count 9.5/0. Average score (5.071428571428571)
Win/lose count 5.0/0. Average score (5.0625)
Win/lose count 11.5/0. Average score (5.777777777777778)
Win/lose count 4.0/0. Average score (5.6)
Final score: 5.6
Test of the FC
Win/lose count 1.0/0. Average score (1.0)
Win/lose count 4.5/1.0. Average score (2.25)
Win/lose count 2.0/0. Average score (2.1666666666666665)
Win/lose count 4.0/0. Average score (2.625)
Win/lose count 2.5/0. Average score (2.6)
Win/lose count 2.5/0. Average score (2.5833333333333335)
Win/lose count 3.5/1.0. Average score (2.5714285714285716)
Win/lose count 1.5/0. Average score (2.4375)
Win/lose count 0.5/0. Average score (2.2222222222222223)
Win/lose count 1.0/0. A

In [0]:
HTML(display_videos('cnn_test9.mp4'))

In [0]:
HTML(display_videos('fc_test9.mp4'))


* We notice that the agent doesn't explore the map well and sometimes get stuck in a position, when there is no positive reward within his visibility, especially for the agent with the fully connected network.

* When we increase the temperature, positive rewards cells get numerous, which helps the agent explore the map since more positive cells gets in his visibility, and therefore the final average score gets higher.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent, env, epoch, prefix=''):
    score, loss, step = 0, 0, 0
    eps_start = agent.epsilon
    eps_final, decay_rate = 0.1, 0.0005

    for e in range(epoch):
        state = env.reset() # We refresh the game at each begin of an epoch
        game_over = False # We assume that the game will finish

        win, lose = 0, 0
        
        while not game_over:
            action = agent.act(state)
            eps = eps_final + (eps_start - eps_final) * np.exp(-step * decay_rate)
            step += 1
            agent.set_epsilon(eps)
            
            # Act action and get reward and new state
            prev_state = state
            state, reward, game_over = env.act(action)
            reward -= env.malus_position[env.x, env.y]

            # Update the counters
            if reward > 0:
                win += reward
            elif reward < 0:
                lose -= reward

            # Apply reinforcement function
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if (e + 1) % 10 == 0:
            env.draw(prefix+str(e + 1))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.1f}/{:.1f} ({}) | explore_proba {:.2f}"
              .format(e, epoch, loss, win, lose, win-lose, eps))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

class EnvironmentExploring(object):
    
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        # Board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)) 
        # Coordinate
        self.x = 0
        self.y = 1
        # Time
        self.t = 0
        self.scale=16
        # To draw
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
    
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)
        
    def reset(self):
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]
        # Bonus
        bonus = 0.5 * np.random.binomial(1, self.temperature, size=self.grid_size ** 2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)
        # Malus
        malus = -1.0 * np.random.binomial(1, self.temperature, size=self.grid_size ** 2)
        malus = malus.reshape(self.grid_size, self.grid_size)
        malus[bonus > 0]=0
        # Draw
        self.to_draw = np.zeros((self.max_time + 2, self.grid_size * self.scale, self.grid_size * self.scale, 3))
        # Board
        self.board = bonus + malus
        self.malus_position = 0 * self.board
        # Position
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
       
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state      
     
    def get_frame(self,t):
        b = np.zeros((self.grid_size, self.grid_size, 3)) + 128
        b[self.x,self.y,:]=256
        b[self.board > 0, 0] = 256
        b[self.board < 0, 2] = 256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)
        self.to_draw[t,:,:,:] = b

    def act(self, action,train = False):
        # Get frame
        self.get_frame(int(self.t))
        # Position
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.position[self.x, self.y] = 1
        # Analyze on the action type
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        # Time step
        self.t = self.t + 1
        # Reward
        reward = 0
        if train:
          reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # State
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        return state, reward, game_over

In [0]:
# Training
size = 13
T=200
temperature=0.3
epochs_train=150
epochs_test=20

env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_cnn = DQN_CNN(size, lr=1e-3, epsilon = 0.9, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent_cnn, env, 150, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore90.mp4'))

Epoch 000/150 | Loss 0.0174 | Win/lose count 6.4/39.5 (-33.10000000000013) | explore_proba 0.82
Epoch 001/150 | Loss 0.0128 | Win/lose count 8.0/38.1 (-30.10000000000015) | explore_proba 0.75
Epoch 002/150 | Loss 0.0101 | Win/lose count 6.0/31.6 (-25.60000000000013) | explore_proba 0.69
Epoch 003/150 | Loss 0.0080 | Win/lose count 15.2/29.3 (-14.100000000000088) | explore_proba 0.64
Epoch 004/150 | Loss 0.0069 | Win/lose count 3.6/28.2 (-24.600000000000087) | explore_proba 0.58
Epoch 005/150 | Loss 0.0067 | Win/lose count 6.0/25.6 (-19.60000000000011) | explore_proba 0.54
Epoch 006/150 | Loss 0.0043 | Win/lose count 14.0/26.6 (-12.600000000000108) | explore_proba 0.50
Epoch 007/150 | Loss 0.0753 | Win/lose count 12.4/26.0 (-13.600000000000055) | explore_proba 0.46
Epoch 008/150 | Loss 0.0059 | Win/lose count 10.0/26.6 (-16.60000000000009) | explore_proba 0.42
Epoch 009/150 | Loss 0.0048 | Win/lose count 16.0/26.1 (-10.100000000000051) | explore_proba 0.39
Epoch 010/150 | Loss 0.0080 | 

In [0]:
# Evaluation
test(agent_cnn,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore15.mp4'))

Win/lose count 12.5/1.0. Average score (11.5)
Win/lose count 20.0/1.0. Average score (15.25)
Win/lose count 16.0/0. Average score (15.5)
Win/lose count 16.5/1.0. Average score (15.5)
Win/lose count 6.0/0. Average score (13.6)
Win/lose count 10.0/0. Average score (13.0)
Win/lose count 15.5/1.0. Average score (13.214285714285714)
Win/lose count 20.5/1.0. Average score (14.0)
Win/lose count 11.0/1.0. Average score (13.555555555555555)
Win/lose count 15.5/0. Average score (13.75)
Win/lose count 16.5/1.0. Average score (13.909090909090908)
Win/lose count 12.0/0. Average score (13.75)
Win/lose count 0/0. Average score (12.692307692307692)
Win/lose count 6.0/0. Average score (12.214285714285714)
Win/lose count 1.0/0. Average score (11.466666666666667)
Win/lose count 16.0/0. Average score (11.75)
Win/lose count 6.5/0. Average score (11.441176470588236)
Win/lose count 5.0/0. Average score (11.083333333333334)
Win/lose count 13.5/0. Average score (11.210526315789474)
Win/lose count 3.0/0. Averag

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***